# Record & Replay Tutorial

This tutorial demonstrates PyTestLab's **Record & Replay** functionality, which enables you to:

- 🎯 Create reproducible measurement procedures
- 🛡️ Validate measurement script integrity
- 🔬 Run measurements offline without hardware
- 🧪 Implement regression testing for lab procedures

We'll walk through recording a real measurement session and then replaying it with validation.

## Overview

The replay system works in two phases:

### 1. Recording Phase
- Your script runs against **real instruments**
- The `SessionRecordingBackend` captures every SCPI command and response
- Results are stored in a structured YAML session file

### 2. Replay Phase  
- Your script runs against the `ReplayBackend`
- Each command is validated against the recorded sequence
- Recorded responses are returned exactly as captured
- Any deviation triggers a `ReplayMismatchError`

## Setup

Let's start by importing the necessary modules and setting up our measurement environment.

In [ ]:
import asyncio
import yaml
from pathlib import Path

from pytestlab.instruments import AutoInstrument
from pytestlab.instruments.backends import ReplayBackend, SessionRecordingBackend
from pytestlab.instruments.backends.errors import ReplayMismatchError
from pytestlab import Bench

print("PyTestLab Record & Replay Tutorial")
print("=" * 40)

## Part 1: Recording a Session

First, let's create a simple measurement procedure that we want to record. This example uses a simulated power supply, but in practice you would use real hardware.

In [ ]:
async def power_supply_characterization(psu):
    """Example measurement procedure to be recorded."""
    print("🔬 Starting power supply characterization...")
    
    # Get instrument ID
    device_id = await psu.id()
    print(f"📋 Device: {device_id}")
    
    # Set current limit for safety
    await psu.set_current(1, 0.1)  # 100mA limit
    print("⚡ Current limit set to 100mA")
    
    # Enable output
    await psu.output(1, True)
    print("🔛 Output enabled")
    
    # Voltage sweep
    voltages = [1.0, 2.0, 3.0, 4.0, 5.0]
    measurements = []
    
    for voltage in voltages:
        print(f"📊 Setting voltage to {voltage}V")
        await psu.set_voltage(1, voltage)
        
        # Wait for settling
        await asyncio.sleep(0.1)
        
        # Read back values
        measured_v = await psu.read_voltage(1)
        measured_i = await psu.read_current(1)
        
        measurements.append({
            'set_voltage': voltage,
            'measured_voltage': measured_v,
            'measured_current': measured_i
        })
        
        print(f"  📈 Measured: {measured_v:.3f}V, {measured_i:.3f}A")
    
    # Clean shutdown
    await psu.output(1, False)
    await psu.set_voltage(1, 0.0)
    print("🔒 Output disabled and voltage reset")
    
    return measurements

Now let's record this procedure. In a real scenario, you would use actual instrument addresses, but here we'll use simulation for demonstration:

In [ ]:
async def record_measurement_session():
    """Record the measurement procedure."""
    print("🎬 RECORDING PHASE")
    print("-" * 20)
    
    # Create PSU with simulation (in practice, use real instrument address)
    psu = AutoInstrument.from_config("keysight/EDU36311A", simulate=True)
    
    # Wrap with SessionRecordingBackend to capture interactions
    session_file = "tutorial_session.yaml"
    recording_backend = SessionRecordingBackend(
        psu._backend, 
        session_file,
        instrument_key="psu"
    )
    psu._backend = recording_backend
    
    await psu.connect_backend()
    
    try:
        # Run the measurement (will be recorded)
        results = await power_supply_characterization(psu)
        print(f"\n✅ Recording complete! Session saved to {session_file}")
        return results
    finally:
        await psu.close()

# Record the session
recorded_results = await record_measurement_session()

Let's examine the recorded session file to understand its structure:

In [ ]:
# Load and display the session file
with open("tutorial_session.yaml") as f:
    session_data = yaml.safe_load(f)

print("📄 Session File Structure:")
print(yaml.dump(session_data, default_flow_style=False, indent=2)[:1000] + "...")

# Count recorded commands
command_count = len(session_data['psu']['log'])
print(f"\n📊 Total commands recorded: {command_count}")

# Show command types
command_types = {}
for entry in session_data['psu']['log']:
    cmd_type = entry['type']
    command_types[cmd_type] = command_types.get(cmd_type, 0) + 1

print("📈 Command breakdown:")
for cmd_type, count in command_types.items():
    print(f"  {cmd_type}: {count}")

## Part 2: Replaying the Session

Now let's replay the exact same measurement procedure using the recorded session:

In [ ]:
async def replay_measurement_session():
    """Replay the recorded measurement procedure."""
    print("\n🎭 REPLAY PHASE")
    print("-" * 20)
    
    # Create ReplayBackend using the recorded session
    replay_backend = ReplayBackend("tutorial_session.yaml")
    
    # Create PSU with replay backend
    psu = AutoInstrument.from_config(
        "keysight/EDU36311A",
        backend_override=replay_backend
    )
    
    await psu.connect_backend()
    
    try:
        # Run the exact same measurement (will replay recorded responses)
        results = await power_supply_characterization(psu)
        print("\n✅ Replay successful! All commands matched the recording.")
        return results
    finally:
        await psu.close()

# Replay the session
replayed_results = await replay_measurement_session()

Let's verify that the replayed results exactly match the recorded ones:

In [ ]:
print("🔍 RESULT COMPARISON")
print("-" * 20)

print(f"Recorded results: {len(recorded_results)} measurements")
print(f"Replayed results: {len(replayed_results)} measurements")

all_match = True
for i, (orig, replay) in enumerate(zip(recorded_results, replayed_results)):
    v_match = abs(orig['measured_voltage'] - replay['measured_voltage']) < 1e-6
    i_match = abs(orig['measured_current'] - replay['measured_current']) < 1e-6
    
    if v_match and i_match:
        print(f"✅ Step {i+1}: Perfect match")
    else:
        print(f"❌ Step {i+1}: Mismatch detected!")
        all_match = False

if all_match:
    print("\n🎉 Perfect replay! All measurements exactly match the recording.")
else:
    print("\n⚠️  Some measurements don't match - check your replay setup.")

## Part 3: Validation and Error Detection

Now let's see what happens when we try to deviate from the recorded sequence:

In [ ]:
async def test_sequence_validation():
    """Demonstrate sequence validation by intentionally deviating."""
    print("\n🚨 VALIDATION TESTING")
    print("-" * 20)
    
    replay_backend = ReplayBackend("tutorial_session.yaml")
    psu = AutoInstrument.from_config(
        "keysight/EDU36311A",
        backend_override=replay_backend
    )
    
    await psu.connect_backend()
    
    try:
        # Start normally
        device_id = await psu.id()
        print(f"✅ Device ID: {device_id}")
        
        await psu.set_current(1, 0.1)
        print("✅ Current limit set correctly")
        
        await psu.output(1, True)
        print("✅ Output enabled correctly")
        
        # Now try to deviate from the recorded sequence
        print("\n⚠️  Attempting to deviate from recorded sequence...")
        print("   (Recording expects 1.0V, but we'll try 2.5V)")
        
        # This should trigger a ReplayMismatchError
        await psu.set_voltage(1, 2.5)  # Recording expects 1.0V first!
        
    except ReplayMismatchError as e:
        print(f"\n🛡️  VALIDATION SUCCESS: {e}")
        print("   The replay system correctly detected the sequence deviation!")
    except Exception as e:
        print(f"\n❌ Unexpected error: {e}")
    finally:
        await psu.close()

await test_sequence_validation()

## Part 4: CLI Workflow

PyTestLab also provides convenient CLI commands for record and replay operations:

In [ ]:
# Create a measurement script file for CLI demonstration
script_content = '''
#!/usr/bin/env python3
"""CLI measurement script for replay demonstration."""

import asyncio

async def main(bench):
    """Main measurement function called by PyTestLab CLI."""
    psu = bench.psu
    
    print("Starting CLI measurement...")
    
    # Simple voltage measurement
    await psu.set_current(1, 0.05)
    await psu.output(1, True)
    await psu.set_voltage(1, 3.3)
    
    voltage = await psu.read_voltage(1)
    current = await psu.read_current(1)
    
    await psu.output(1, False)
    
    print(f"Measured: {voltage}V, {current}A")
    return {"voltage": voltage, "current": current}
    
if __name__ == "__main__":
    print("Use: pytestlab replay run cli_measurement.py --session session.yaml")
'''

with open("cli_measurement.py", "w") as f:
    f.write(script_content)

print("📝 Created CLI measurement script: cli_measurement.py")

In [ ]:
# Create a simple bench configuration
bench_config = '''
bench_name: "Tutorial Bench"
simulate: true
instruments:
  psu:
    profile: "keysight/EDU36311A"
    address: "TCPIP0::192.168.1.100::inst0::INSTR"
'''

with open("tutorial_bench.yaml", "w") as f:
    f.write(bench_config)

print("📝 Created bench configuration: tutorial_bench.yaml")

In [ ]:
print("🖥️  CLI COMMAND EXAMPLES:")
print("=" * 25)
print()
print("1️⃣  Record a measurement session:")
print("   pytestlab replay record cli_measurement.py --bench tutorial_bench.yaml --output cli_session.yaml")
print()
print("2️⃣  Replay the recorded session:")
print("   pytestlab replay run cli_measurement.py --session cli_session.yaml")
print()
print("3️⃣  View replay command help:")
print("   pytestlab replay --help")
print()
print("📂 Files created for CLI workflow:")
print("   - cli_measurement.py (measurement script)")
print("   - tutorial_bench.yaml (bench configuration)")
print("   - tutorial_session.yaml (recorded session)")

## Part 5: Best Practices

Here are some key recommendations for effective use of the replay system:

In [ ]:
print("💡 BEST PRACTICES")
print("=" * 17)
print()
print("🎯 Recording Phase:")
print("   • Always record with real instruments for authentic responses")
print("   • Use descriptive session file names (e.g., 'psu_characterization_v2.1.yaml')")
print("   • Include version information in session metadata")
print("   • Test your recording immediately after creation")
print()
print("🔄 Replay Phase:")
print("   • Use identical instrument profiles during recording and replay")
print("   • Handle ReplayMismatchError gracefully in production code")
print("   • Enable debug logging to troubleshoot sequence issues")
print()
print("📁 File Management:")
print("   • Store session files in version control")
print("   • Organize sessions by measurement type or instrument")
print("   • Include README files documenting session purposes")
print()
print("🧪 Testing:")
print("   • Create regression tests using recorded sessions")
print("   • Validate session integrity periodically")
print("   • Use replay mode for CI/CD pipeline testing")

## Summary

In this tutorial, we've explored PyTestLab's Record & Replay functionality:

✅ **Recording**: Captured real instrument interactions using `SessionRecordingBackend`  
✅ **Replaying**: Validated exact sequence reproduction using `ReplayBackend`  
✅ **Validation**: Demonstrated automatic detection of sequence deviations  
✅ **CLI Integration**: Showed command-line workflow for production use  
✅ **Best Practices**: Covered recommendations for effective replay system usage  

### Key Benefits Achieved:

- **🎯 Reproducibility**: Exact measurement sequences can be repeated indefinitely
- **🛡️ Integrity**: Scripts cannot accidentally deviate from validated procedures  
- **🔬 Offline Analysis**: Complex measurements run without physical instruments
- **🧪 Regression Testing**: Automated validation of measurement procedure consistency

### Next Steps:

1. Try recording your own measurement procedures with real instruments
2. Integrate replay validation into your testing workflows  
3. Explore multi-instrument session recording for complex setups
4. Set up automated regression testing using recorded sessions

The Record & Replay system provides a powerful foundation for reproducible, validated measurement automation in PyTestLab! 🚀